In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wrapper)
# result = wiki_tool.run("What is LangChain?")
# print(result)
# type(result)

In [ ]:
llm = ChatOllama(model="gpt-oss:20b", temperature=0.3)
employee_dep= {"manuf":"Manufacturing", "qc":"Quality control", "wh":"Warehouse", "hk":"Housekeeping", "gen":"General"}

In [ ]:
def check_employee_department(emp_id: str):
    """Check the only employee Department with employee id
       Args:
          emp_id: first str
    """
    try:
        result = ""
        #result=next(e_dep if eid in emp_id else f"{employee_dep["gen"]}" for eid, e_dep in employee_dep.items())
        for eid, e_dep in employee_dep.items():
            if eid in emp_id:
                result = e_dep
                break
    except Exception as e:
        result = f"Error: {e}"

    if not result:
        result = f"{employee_dep["gen"]}"

    print(result)
    return result


llm_with_tools = llm.bind_tools([check_employee_department, wiki_tool])

In [ ]:
msg = [HumanMessage(content="What is the department name of employee qc980")]
response  = [llm_with_tools.invoke(msg)]
print(response)
for msg in response:
    msg.pretty_print()

In [ ]:
class MsgState(MessagesState):
    pass

In [ ]:
sys_msg = SystemMessage(content="You are the intelligent AI system for the queries use Wiki tool and get the result and summarise that in a shorter and precise way, if the question is related to employee department then only use the employee department tool and get the result")
def tool_calling(state:MsgState):
    return {"messages":[llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [ ]:
builder_sg = StateGraph(MsgState)
builder_sg.add_node("tool_call_llm",tool_calling)
builder_sg.add_edge(START, "tool_call_llm")
builder_sg.add_edge("tool_call_llm", END)

graph = builder_sg.compile()

In [ ]:
mg = {"messages":HumanMessage(content="What is the department name of employee wh980")}
resp=graph.invoke(mg)
resp
for msg in resp["messages"]:
    msg.pretty_print()

In [ ]:
workflow = StateGraph(MsgState)
workflow.add_node("tool_call_llm", tool_calling)
workflow.add_node("tools", ToolNode([check_employee_department, wiki_tool]))

workflow.add_edge(START, "tool_call_llm")
workflow.add_conditional_edges("tool_call_llm", tools_condition)
workflow.add_edge("tools", "tool_call_llm")

wf_graph = workflow.compile()
wf_graph

In [ ]:
#mg = {"messages":[HumanMessage(content="Quick comparison on Python and Kotlin programming languages & what is the department for the employee id hk782")]}
mg = {"messages":[HumanMessage(content="what is the department of employee id with hk782? and quick comparison on india and usa work culture in 50 words")]}
#mg1 = {"messages":[SystemMessage(content="You are the intelligent AI system invoke tools for the employee related things"), HumanMessage(content="what is the department for the employee id hk782")]}
resp=wf_graph.invoke(mg)
op = resp["messages"]
for msg in op:
    msg.pretty_print()